In [1]:
import os
import pickle
import numpy as np
import re
import emoji
import regex

In [2]:
def extract_emojis(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

In [3]:
def delete_emojis(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            
            text = re.sub("|".join(word), "", text)

    return text

In [4]:
# load emoji list
fullfile = os.path.expanduser("~/Dropbox/insight/Emoji/"+'mySmileys.p')
with open(fullfile, 'rb') as fp:
    emoji_list = pickle.load(fp)
print(emoji_list)
print(len(emoji_list))

['😀', '😁', '😂', '🤣', '😃', '😄', '😅', '😆', '😉', '😊', '😋', '😎', '😍', '😘', '😗', '😙', '😚', '☺️', '🙂', '🤗', '🤔', '😐', '😑', '😶', '🙄', '😏', '😣', '😥', '😮', '🤐', '😯', '😪', '😫', '😴', '😌', '😛', '😜', '😝', '🤤', '😒', '😓', '😔', '😕', '🙃', '🤑', '😲', '☹️', '🙁', '😖', '😞', '😟', '😤', '😢', '😭', '😦', '😧', '😨', '😩', '😬', '😰', '😱', '😳', '😵', '😡', '😠', '😷', '🤒', '🤕', '🤢', '🤧', '😇', '🤠', '🤡', '🤥', '🤓']
75


## 1. Testing: word-to-vec with TfidfVectorizer

In [5]:
def sanitize_tweets(tweet):
    # delete twitter-specific tags
    b1 = re.findall(r'@\S*', tweet) # Twitter user names
    b2 = re.findall(r'RT\S*', tweet) # "RT"
    b3 = re.findall(r'http\S*', tweet) # links

    c = tweet #list(tweet.split())
    for b in b1:
        c = re.sub(re.escape(b),"",c)
    for b in b2:
        c = re.sub(re.escape(b),"",c)
    for b in b3:
        c = re.sub(re.escape(b),"",c)
    return c
    
tweet = "RT @aa RT @bbb https:__ and other words! httpsaa" #RawT[-1]
sanitize_tweets(tweet)

'     and other words! '

In [17]:
# TASK: Build a vectorizer that splits strings into sequence of 1 to 3
# characters instead of word tokens
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='word',# min_df=3, ngram_range=(1, 3)
                             max_features = 2000, #stop_words='english',
                             token_pattern=r"(?u)\b\w\w+\b|!|\?|\"|\'")

# https://stackoverflow.com/questions/39254134/how-to-preserve-punctuation-marks-in-scikit-learn-text-countvectorizer-or-tfidfv

In [16]:
fullfile = os.path.expanduser("~/Dropbox/insight_datadir/5k/"+'outfile'+'😱'+'.p')
with open(fullfile, 'rb') as fp:
    Tweets = pickle.load(fp)

In [10]:
tweet = "😍 ❤️🎶🎤🎂🎉🎁🎈😍🌟💫 shsha;lsdij "
emojis = extract_emojis(tweet)
print(emojis)
a = set(emojis) & set(emoji_list)
print(len(a))

['😍', '❤️', '🎶', '🎤', '🎂', '🎉', '🎁', '🎈', '😍', '🌟', '💫']
1


In [21]:
count = 0
id = 0
for tweet in Tweets[:100]:
    emojis = extract_emojis(tweet)
    a = set(emojis) & set(emoji_list)
    if len(a)==1:
        print(tweet + "".join(emojis))
        print("----------------")
        count+=1
    id += 1
print(count)

RT @aMrazing: HOLYMOTHER OF ALL TWISTS OMFG NAGINI WAS A WOMAN LIKE WHAT THE HELL 😱😱😱😱😱
KALIAN TONTON DONG TRAILER BARU FANTASTIC BEASTS TE…😱😱😱😱😱
----------------
@mizz_nayomie Your iPhone😱😱
----------------
RT @ChiSportUpdates: 😱CHICAGOOOOOO😱

(Picture via Chicago Bulls) https://t.co/gpqmogYaYP😱😱
----------------
RT @dinahjane97: 😱😱😱😱😱😱😱😱😱😱 OH MAhhh GAHHH!!! LOVE LOVE LOVE YOU GUYS X glad y’all diggin it 🍾 https://t.co/vXIuOfdUsy😱😱😱😱😱😱😱😱😱😱🍾
----------------
RT @dodo: These people CANNOT believe the wild animals they’re running into 😱 https://t.co/vE0MiJaoG7😱
----------------
RT @FortniteFunny: Holy!! 😱😱

Credit @ObeyGorb https://t.co/yzWUs0YZIO😱😱
----------------
RT @FortniteFunny: Holy!! 😱😱

Credit @ObeyGorb https://t.co/yzWUs0YZIO😱😱
----------------
RT @OnlineMagazin: 🆘‼😱🚖 #Spain: Ouch! Oh dear! On the Spanish island of Ibiza, an African physicist was hit by a racist Spanish taxi driver…🆘‼😱🚖
----------------
RT @FortniteFunny: Holy!! 😱😱

Credit @ObeyGorb https://t.co/yzWUs0YZIO😱😱
--

In [14]:
print(Tweets[11])
print(Tweets[13])
print(Tweets[27])

RT @WORLDMUSICAWARD: Happy 26th Birthday to #EXO's very handsome and talented #CHEN! #HappyChenDay! @weareoneEXO
❤️🇰🇷🎶🎤🎂🎉🎁🎈😍🌟💫🎇  
https://t…
RT @WORLDMUSICAWARD: Happy 26th Birthday to #EXO's very handsome and talented #CHEN! #HappyChenDay! @weareoneEXO
❤️🇰🇷🎶🎤🎂🎉🎁🎈😍🌟💫🎇  
https://t…
RT @WORLDMUSICAWARD: Happy 26th Birthday to #EXO's very handsome and talented #CHEN! #HappyChenDay! @weareoneEXO
❤️🇰🇷🎶🎤🎂🎉🎁🎈😍🌟💫🎇  
https://t…


In [7]:

fullfile = os.path.expanduser("~/Dropbox/insight_datadir/5k/"+'outfile'+'😮'+'.p')
with open(fullfile, 'rb') as fp:
    Tweets = pickle.load(fp)
T = []
T3 = ""
for tweet in Tweets:
#     emojis = extract_emojis(tweet)
#     text = re.sub("|".join(emoji_list), "", tweet) # delete emojis from tweet
    text = delete_emojis(tweet)
    text = sanitize_tweets(text)
    T.append(text)
    T3 = T3 + text
    

In [ ]:
# example

In [29]:
S1 = "The car is driven on the road."

S2 = "The truck is driven on the highway."

In [35]:
# sanity check generated vocab here
X = vectorizer.fit_transform([S1,S2])

In [36]:
# EDA
f_names = vectorizer.get_feature_names()
print(f_names)

['car', 'driven', 'highway', 'is', 'on', 'road', 'the', 'truck']


In [38]:
print(X)
print(X.shape)

  (0, 6)	0.6043795515372431
  (0, 0)	0.42471718586982765
  (0, 3)	0.30218977576862155
  (0, 1)	0.30218977576862155
  (0, 4)	0.30218977576862155
  (0, 5)	0.42471718586982765
  (1, 6)	0.6043795515372431
  (1, 3)	0.30218977576862155
  (1, 1)	0.30218977576862155
  (1, 4)	0.30218977576862155
  (1, 7)	0.42471718586982765
  (1, 2)	0.42471718586982765
(2, 8)


In [31]:
# (temp)
# print(X.max())
# IX = np.nonzero(X[1,:]>0.1)
# type(IX)
# print(IX)
# print(f_names[487])

25
(2, 25)
0.37930349280874964
(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 15, 18, 20, 21, 22, 23, 24],
      dtype=int32))


IndexError: list index out of range

## 2. Make 75-dim target vector

In [18]:
N = []
for ii in range(75):
    N.append(ii)
Edict = dict(zip(emoji_list, N))
print(Edict)

{'😀': 0, '😁': 1, '😂': 2, '🤣': 3, '😃': 4, '😄': 5, '😅': 6, '😆': 7, '😉': 8, '😊': 9, '😋': 10, '😎': 11, '😍': 12, '😘': 13, '😗': 14, '😙': 15, '😚': 16, '☺️': 17, '🙂': 18, '🤗': 19, '🤔': 20, '😐': 21, '😑': 22, '😶': 23, '🙄': 24, '😏': 25, '😣': 26, '😥': 27, '😮': 28, '🤐': 29, '😯': 30, '😪': 31, '😫': 32, '😴': 33, '😌': 34, '😛': 35, '😜': 36, '😝': 37, '🤤': 38, '😒': 39, '😓': 40, '😔': 41, '😕': 42, '🙃': 43, '🤑': 44, '😲': 45, '☹️': 46, '🙁': 47, '😖': 48, '😞': 49, '😟': 50, '😤': 51, '😢': 52, '😭': 53, '😦': 54, '😧': 55, '😨': 56, '😩': 57, '😬': 58, '😰': 59, '😱': 60, '😳': 61, '😵': 62, '😡': 63, '😠': 64, '😷': 65, '🤒': 66, '🤕': 67, '🤢': 68, '🤧': 69, '😇': 70, '🤠': 71, '🤡': 72, '🤥': 73, '🤓': 74}


In [9]:
# # test code: make 75-D target
# target_arr = np.zeros((10, 75))
# for ii in range(10):
#     e_list = D.raw_target[ii+10]
#     for x in e_list:
#         if x in emoji_list:
#             ix = Edict[x]
#             target_arr[ii,ix] = 1
# #D.target = target_arr

In [19]:
# 75-D preprecessing
# IN
target_names = ['😍','😡'] # emoji_list 

# OUT
rawdata = []
data = []
raw_target = []
Len = []
filenames = []
target_arr = np.zeros((5000*75, 75))
search_emoji = []

# main loop
ii = 0
for keyword in target_names:
    print(keyword)
    fullfile = os.path.expanduser("~/Dropbox/insight_datadir/5k/"+'outfile'+keyword+'.p')
    with open(fullfile, 'rb') as fp:
        Tweets = pickle.load(fp)
        Len.append(len(Tweets))
        
        Text = []
        RawText = []
        E = []
        for tweet in Tweets:
            # save raw tweet
            RawText.append(tweet)
            # take out all emojis from input tweets
            emojis = extract_emojis(tweet)
            E.append(emojis)
            
            # save emojis into target array 
            for x in emojis:
                if x in emoji_list:
                    ix = Edict[x]
                    target_arr[ii,ix] = 1
            
            # save sanitized text
            text = sanitize_tweets(tweet)
            text = delete_emojis(text)
            Text.append(text)
            ii = ii+1
            
        data = data + Text
        rawdata = rawdata + RawText
        raw_target = raw_target + E
        filenames = filenames + [fullfile]
        search_emoji = search_emoji + [keyword]

😀
😁
😂
🤣
😃
😄
😅
😆
😉
😊
😋
😎
😍
😘
😗
😙
😚
☺️
🙂
🤗
🤔
😐
😑
😶
🙄
😏
😣
😥
😮
🤐
😯
😪
😫
😴
😌
😛
😜
😝
🤤
😒
😓
😔
😕
🙃
🤑
😲
☹️
🙁
😖
😞
😟
😤
😢
😭
😦
😧
😨
😩
😬
😰
😱
😳
😵
😡
😠
😷
🤒
🤕
🤢
🤧
😇
🤠
🤡
🤥
🤓


In [20]:
# optional now: get 1-D target
numEmojis = len(target_names)
arr = []
for i in range(numEmojis):
    arr.extend([i] * Len[i]) 
target_1d = np.array(arr, dtype=int)

In [45]:
# this struct is saving relatively raw data, for later use ~ visualizations
class tweet_data:
    pass

D = tweet_data()
D.raw_data = rawdata
D.data = data
D.raw_target = raw_target
D.filenames = filenames
D.numTweets = Len
D.target_1d = target_1d
D.target_arr = target_arr

In [46]:
len(data)


375000

In [47]:
fullfile = os.path.expanduser("~/Dropbox/insight/Twitter/"+'tweet_data_75x5k.p')
with open(fullfile, 'wb') as fp:
    pickle.dump(D, fp)

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target_arr, test_size= 0.2, random_state=1)

NameError: name 'data' is not defined

In [23]:
# this will be directly loaded to train model
class train_data:
    pass

T = train_data()
T.data = data
# X = vectorizer.fit_transform(data)
# T.X = X
T.feature_names = vectorizer.get_feature_names()
# T.X_shape = X.shape
T.Y = target_arr
T.y_1d = target_1d
T.target_names = target_names

In [24]:


fullfile = os.path.expanduser("~/Dropbox/insight/Twitter/"+'train_data_75x5k.p')
with open(fullfile, 'wb') as fp:
    pickle.dump(T, fp)


### (old) small model with only 2 opposing emojis

In [16]:
# # 1-D preprecessing
# # IN
# target_names = ['😍','😡'] # emoji_list # 

# # OUT
# rawdata = []
# data = []
# raw_target = []
# Len = []
# filenames = []

# # main loop
# for keyword in target_names:
#     fullfile = os.path.expanduser("~/Dropbox/insight_datadir/5k/"+'outfile'+keyword+'.p')
#     with open(fullfile, 'rb') as fp:
#         itemlist = pickle.load(fp)
#         Len.append(len(itemlist))
        
#         T = []
#         RawT = []
#         E = []
#         for tweet in itemlist:
#             # save raw tweet
#             RawT.append(tweet)
#             # take out all emojis from input tweets
#             emojis = extract_emojis(tweet)
#             E.append(emojis)
#             # save sanitized text
#             text = re.sub("|".join(emoji_list), "", tweet)
#             T.append(text)
#         data = data + T
#         rawdata = rawdata + RawT
#         raw_target = raw_target + E
#         filenames = filenames + [fullfile]
        

# class tweet_data:
#     pass

# D = tweet_data()
# D.raw_data = rawdata
# D.data = data
# D.raw_target = raw_target
# D.filesnames = filenames
# D.numTweets = Len

# # fullfile = os.path.expanduser("~/Dropbox/insight/Twitter/"+'tweets_2x5k.p')
# # #fullfile = os.path.expanduser("~/Dropbox/insight/Twitter/"+'tweets_75x5k.p')
# # with open(fullfile, 'wb') as fp:
# #     pickle.dump(D, fp)


# class tweet_train:
#     pass
# T = tweet_train()
# T.target_names = target_names
# T.data = D.data
# T.filenames = filenames
# T.target = target

# fullfile = os.path.expanduser("~/Dropbox/insight/Twitter/"+'tweet_train_2x5k.p')
# with open(fullfile, 'wb') as fp:
#     pickle.dump(T, fp)


In [22]:
import datetime
datetime.now()

AttributeError: module 'datetime' has no attribute 'now'